<a href="https://colab.research.google.com/github/aderdouri/ql_web_app/blob/master/ql_notebooks/americanOptionTestBaroneAdesiWhaleyValues.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install QuantLib-Python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 25.3 MB/s eta 0:00:00


In [ ]:
import QuantLib as ql
import sys

# --- Configuration & Data ---

# Determine if running in interactive environment (like Jupyter) to adjust output
is_interactive = hasattr(sys, 'ps1')

# American option test data (same as provided)
# Columns: type, strike, spot, q (dividend yield), r (risk-free rate), t (time to maturity in years), vol (volatility), expected_result
american_option_data = [
    # type,         strike, spot,   q,    r,    t,    vol,  expected
    (ql.Option.Call, 100.0,  90.0, 0.10, 0.10, 0.10, 0.15,  0.0206),
    (ql.Option.Call, 100.0, 100.0, 0.10, 0.10, 0.10, 0.15,  1.8771),
    (ql.Option.Call, 100.0, 110.0, 0.10, 0.10, 0.10, 0.15, 10.0089),
    (ql.Option.Put,  100.0,  90.0, 0.10, 0.10, 0.10, 0.15, 10.0000),
    (ql.Option.Put,  100.0, 100.0, 0.10, 0.10, 0.10, 0.15,  1.8770),
    (ql.Option.Put,  100.0, 110.0, 0.10, 0.10, 0.10, 0.15,  0.0410),
]

# --- Global Settings ---

# Set evaluation date (using a fixed date for reproducibility instead of todaysDate())
# calculation_date = ql.Date.todaysDate()
calculation_date = ql.Date(15, 5, 2023) # Example fixed date
ql.Settings.instance().evaluationDate = calculation_date

# Define Day Counter and Calendar
day_counter = ql.Actual360()
calendar = ql.NullCalendar() # Using NullCalendar as holidays are not relevant here

# Define Tolerance for comparison
tolerance = 3.0e-3 # 0.003

# --- Market Data Setup (using Handles for dynamic updates) ---

# Create SimpleQuote objects to hold the values that will change in the loop
spot_quote = ql.SimpleQuote(0.0)
q_rate_quote = ql.SimpleQuote(0.0) # Dividend yield
r_rate_quote = ql.SimpleQuote(0.0) # Risk-free rate
vol_quote = ql.SimpleQuote(0.0)    # Volatility

# Wrap quotes in Handles. Term structures will observe changes through these handles.
spot_handle = ql.QuoteHandle(spot_quote)
q_rate_handle = ql.QuoteHandle(q_rate_quote)
r_rate_handle = ql.QuoteHandle(r_rate_quote)
vol_handle = ql.QuoteHandle(vol_quote)

# Create flat term structures for rates and volatility
# Dividend yield curve
q_yield_term_structure = ql.YieldTermStructureHandle(
    ql.FlatForward(calculation_date, q_rate_handle, day_counter)
)
# Risk-free rate curve
r_yield_term_structure = ql.YieldTermStructureHandle(
    ql.FlatForward(calculation_date, r_rate_handle, day_counter)
)
# Volatility surface (flat in this case)
volatility_term_structure = ql.BlackVolTermStructureHandle(
    ql.BlackConstantVol(calculation_date, calendar, vol_handle, day_counter)
)

# --- Test Execution Loop ---

print("--- American Option Pricing Test (Barone-Adesi/Whaley) ---")
print(f"Evaluation Date: {calculation_date.ISO()}")
print(f"Tolerance: {tolerance}")
print("-" * 60)

for i, data in enumerate(american_option_data):
    opt_type, strike, s, q, r, t, v, expected = data

    # 1. Update Market Data Quotes (Term Structures update automatically via Handles)
    spot_quote.setValue(s)
    q_rate_quote.setValue(q)
    r_rate_quote.setValue(r)
    vol_quote.setValue(v)

    # 2. Define Option Specifics
    payoff = ql.PlainVanillaPayoff(opt_type, strike)

    # Calculate maturity date based on time 't' in years
    # Note: `int(t * 360)` is a crude approximation as used in the original description.
    # Using ql.Period might be more robust for different day counters if needed.
    # maturity_date = calculation_date + ql.Period(int(round(t*365)), ql.Days) # Alternative
    maturity_date = calculation_date + int(t * 360)

    exercise = ql.AmericanExercise(calculation_date, maturity_date)

    # 3. Create Stochastic Process
    # Black-Scholes-Merton process using the handles to the market data
    bsm_process = ql.BlackScholesMertonProcess(
        spot_handle,
        q_yield_term_structure,
        r_yield_term_structure,
        volatility_term_structure
    )

    # 4. Define the Pricing Engine
    # Using Barone-Adesi/Whaley approximation engine for American options
    engine = ql.BaroneAdesiWhaleyApproximationEngine(bsm_process)
    # Other engines like Binomial (e.g., ql.BinomialVanillaEngine(bsm_process, "crr", 100))
    # or Finite Difference (e.g., ql.FdBlackScholesVanillaEngine(bsm_process)) could also be used.

    # 5. Create the Option Instrument
    american_option = ql.VanillaOption(payoff, exercise)

    # 6. Assign the Engine to the Option
    american_option.setPricingEngine(engine)

    # 7. Calculate the Price (NPV)
    calculated_price = american_option.NPV()

    # 8. Compare and Report
    error = abs(calculated_price - expected)

    option_type_str = "Call" if opt_type == ql.Option.Call else "Put"

    # Use rich display for PASS/FAIL in interactive environments
    pass_fail_str = "PASS" if error <= tolerance else "FAIL"

    if is_interactive:
        from IPython.display import display, Markdown
        color = "green" if pass_fail_str == "PASS" else "red"
        md_str = f"**<font color='{color}'>{pass_fail_str}</font>** - {option_type_str:4s} | K={strike:.1f}, S={s:.1f}, q={q:.2f}, r={r:.2f}, T={t:.2f}, vol={v:.2f} | Calc: {calculated_price:.4f}, Exp: {expected:.4f} | Err: {error:.6f}"
        display(Markdown(md_str))
    else: # Standard console output
        print(f"{pass_fail_str:4s} - {option_type_str:4s} | K={strike:<5.1f}, S={s:<5.1f}, q={q:.2f}, r={r:.2f}, T={t:.2f}, vol={v:.2f}", end="")
        print(f" | Calc: {calculated_price:<8.4f}, Exp: {expected:<8.4f} | Error: {error:.6f}")

    # Optional: Print more details on failure
    # if error > tolerance:
    #     print(f"  FAILURE Details:")
    #     print(f"    Input Params: type={option_type_str}, strike={strike}, spot={s}, q={q}, r={r}, t={t}, vol={v}")
    #     print(f"    Expected: {expected:.4f}")
    #     print(f"    Calculated: {calculated_price:.4f}")
    #     print(f"    Error: {error:.6f} (Tolerance: {tolerance})")
    #     print("-" * 20)

print("-" * 60)
print("Test complete.")

--- American Option Pricing Test (Barone-Adesi/Whaley) ---
Evaluation Date: 2023-05-15
Tolerance: 0.003
------------------------------------------------------------


**<font color='green'>PASS</font>** - Call | K=100.0, S=90.0, q=0.10, r=0.10, T=0.10, vol=0.15 | Calc: 0.0206, Exp: 0.0206 | Err: 0.000036

**<font color='green'>PASS</font>** - Call | K=100.0, S=100.0, q=0.10, r=0.10, T=0.10, vol=0.15 | Calc: 1.8769, Exp: 1.8771 | Err: 0.000179

**<font color='green'>PASS</font>** - Call | K=100.0, S=110.0, q=0.10, r=0.10, T=0.10, vol=0.15 | Calc: 10.0061, Exp: 10.0089 | Err: 0.002840

**<font color='green'>PASS</font>** - Put  | K=100.0, S=90.0, q=0.10, r=0.10, T=0.10, vol=0.15 | Calc: 10.0000, Exp: 10.0000 | Err: 0.000000

**<font color='green'>PASS</font>** - Put  | K=100.0, S=100.0, q=0.10, r=0.10, T=0.10, vol=0.15 | Calc: 1.8769, Exp: 1.8770 | Err: 0.000078

**<font color='green'>PASS</font>** - Put  | K=100.0, S=110.0, q=0.10, r=0.10, T=0.10, vol=0.15 | Calc: 0.0410, Exp: 0.0410 | Err: 0.000004

------------------------------------------------------------
Test complete.
